In [0]:
%sql
create catalog if not exists demo_catalog;
create database if not exists demo_catalog.demo_database;

In [0]:
# Config
# Replace with your Event Hub namespace, name, and key
connectionString = "Paste connection string here"
eventHubName = "Type event hub name here"

ehConf = {
  'eventhubs.connectionString' : sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString),
  'eventhubs.eventHubName': eventHubName
}

In [0]:
# Reading stream: Load data from Azure Event Hub into DataFrame 'df' using the previously configured settings
df = spark.readStream \
    .format("eventhubs") \
    .options(**ehConf) \
    .load() \

# Displaying stream: Show the incoming streaming data for visualization and debugging purposes
df.display()

# Writing stream: Persist the streaming data to a Delta table 'streaming.bronze.weather' in 'append' mode with checkpointing
df.writeStream\
    .option("checkpointLocation", "/mnt/streaming/bronze/weather")\
    .outputMode("append")\
    .format("delta")\
    .toTable("streaming.bronze.weather")

#### Silver Layer

Defining the schema for the JSON object.

In [0]:
# # Defining the schema for the JSON object

# json_schema = StructType([
#     StructField("temperature", IntegerType()),
#     StructField("humidity", IntegerType()),
#     StructField("windSpeed", IntegerType()),
#     StructField("windDirection", StringType()),
#     StructField("precipitation", IntegerType()),
#     StructField("conditions", StringType())
# ])

Reading, transforming and writing the stream from the bronze to the silver layer.

In [0]:
# # Reading and Transforming: Load streaming data from the 'streaming.bronze.weather' Delta table, cast 'body' to string, parse JSON, and select specific fields
# df = spark.readStream\
#     .format("delta")\
#     .table("streaming.bronze.weather")\
#     .withColumn("body", col("body").cast("string"))\
#     .withColumn("body",from_json(col("body"), json_schema))\
#     .select("body.temperature", "body.humidity", "body.windSpeed", "body.windDirection", "body.precipitation", "body.conditions", col("enqueuedTime").alias('timestamp'))

# # Displaying stream: Visualize the transformed data in the DataFrame for verification and analysis
# df.display()

# # Writing stream: Save the transformed data to the 'streaming.silver.weather' Delta table in 'append' mode with checkpointing for data reliability
# df.writeStream\
#     .option("checkpointLocation", "/mnt/streaming/silver/weather")\
#     .outputMode("append")\
#     .format("delta")\
#     .toTable("streaming.silver.weather")

#### Gold Layer

Reading, aggregating and writing the stream from the silver to the gold layer.

In [0]:
# # Aggregating Stream: Read from 'streaming.silver.weather', apply watermarking and windowing, and calculate average weather metrics
# df = spark.readStream\
#     .format("delta")\
#     .table("streaming.silver.weather")\
#     .withWatermark("timestamp", "5 minutes") \
#     .groupBy(window("timestamp", "5 minutes")) \
#     .agg(avg("temperature").alias('temperature'), avg("humidity").alias('humidity'), avg("windSpeed").alias('windSpeed'), avg("precipitation").alias('precipitation'))\
# 	.select('window.start', 'window.end', 'temperature', 'humidity', 'windSpeed', 'precipitation')

# # Displaying Aggregated Stream: Visualize aggregated data for insights into weather trends
# df.display()

# # Writing Aggregated Stream: Store the aggregated data in 'streaming.gold.weather_aggregated' with checkpointing for data integrity
# df.writeStream\
#     .option("checkpointLocation", "/mnt/streaming/weather_summary")\
#     .outputMode("append")\
#     .format("delta")\
#     .toTable("streaming.gold.weather_summary")